# Test Function for JackOfAllTradesPSO
---
Description:

- Optimization (min)
- Single-objective
- Constraints (no)
---

The general problem statement is given by:

- $f(\mathbf{x}) = \sum_i |x_i|$, with $x_1 \in \Re$, $x_2 \in [-3, +3]$ and $x_3 \in \{a, b, c\}$,
where D is the corresponding dimension. 

assuming a=0, b=1, c=2, then the global minimum found at: $f(0.0, 0, a) = 0$.

### First we import python libraries and set up the directory of our code.

In [1]:
import os, sys
import numpy as np
from math import fsum

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [2]:
# Import main classes.
from star_pso.auxiliary.swarm import Swarm
from star_pso.auxiliary.utilities import BlockType
from star_pso.auxiliary.data_block import DataBlock
from star_pso.auxiliary.jat_particle import JatParticle
from star_pso.engines.jack_of_all_trades_pso import JackOfAllTradesPSO

### Define the test function.

In [3]:
# Auxiliary mapping.
key_to_value = {"a": 0, "b": 1, "c": 2}

# Test function.
def fun_test1(x: np.typing.ArrayLike):
    
    # Unpack input values.
    x1, x2, x3 = x
    
    # Convert symbols to numerical values.
    x3 = key_to_value[x3]
    
    # Compute the function value.
    f_val = np.sum(np.abs((x1, x2, x3)))

    # Condition for termination.
    solution_found = f_val == 0.0
    
    # Return the solution tuple.
    return -f_val, solution_found
# _end_def_

Here we set the PSO parameters.

In [4]:
# Random number generator.
rng = np.random.default_rng()

# Define the variable set for the categorical
# optimization variable.
var_set = ["a", "b", "c"]

# Define the number of optimizing variables.
D = 3

# Define the number of particles.
N = min(5*D, 100)

# Initial population.
swarm_t0 = Swarm([JatParticle([DataBlock(rng.uniform(-10.0, +10.0),
                                         BlockType.FLOAT,
                                         lower_bound=-10.0,
                                         upper_bound=+10.0),
                               DataBlock(rng.integers(-3, +3, endpoint=True),
                                         BlockType.INTEGER,
                                         lower_bound=-3,
                                         upper_bound=+3),
                               DataBlock(np.ones(D)/D,
                                         BlockType.CATEGORICAL,
                                         valid_set=var_set)]) for _ in range(N)])

# Create the JackOfAllTradesPSO object that will carry on the optimization.
test_PSO = JackOfAllTradesPSO(initial_swarm = swarm_t0, obj_func = fun_test1)

In [5]:
test_PSO.run(max_it = 1000,
             options = {"w": 0.5, "c1": 0.1, "c2": 0.1, "fipso": False},
             reset_swarm=True,
             verbose = True)

Initial f_optimal = -4.6318
Iteration:     1 -> f_optimal = -5.4953
Iteration:   101 -> f_optimal = -0.5144
Iteration:   201 -> f_optimal = -0.2021
Iteration:   301 -> f_optimal = -0.1130
Iteration:   401 -> f_optimal = -1.0955
Iteration:   501 -> f_optimal = -0.0508
Iteration:   601 -> f_optimal = -1.0425
Iteration:   701 -> f_optimal = -0.0285
Iteration:   801 -> f_optimal = -0.0204
Iteration:   901 -> f_optimal = -0.0139
Final f_optimal = -0.0105
run: elapsed time = 4.555 seconds.


### Optimization process.

Here we call the PSO object. We set a number of parameters, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergence, etc.

In [6]:
# Extract the optimal solution from the PSO.
optimal_solution = test_PSO.swarm.best_particle()

# Display the (final) optimal value.
print(f"Optimum Found: {optimal_solution.value}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.position, start=1):
    print(f"x{i} = {xi}")
# _end_for_

print("\nBest sampled position:")
print(test_PSO.stats["x_best"][-1])

Optimum Found: -0.010496228787112214

x1 = 0.010496228787112214
x2 = 0
x3 = [0.41041564 0.30310066 0.28648371]

Best sampled position:
[0.010496228787112214, 0, 'a']


### End of file